In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## moving the data 

**이것을 이제 module화 시켜서, 다양한 tar file, self.glob에 대해서 해서 여러개의 file들을 평행하게 (using multiproceissing or sth) 만들어서 돌리기!)**

In [2]:
import tarfile
import os 
import shutil
import glob 
import subprocess

class Args():
    def __init__(self):
        self.T1_DWI_dir = "/scratch/08834/tg881334/NDARINV0.tar"
        self.T2_dir = "/scratch/08834/tg881334/NDARINV0-5.tar"
        self.save_dir = "/scratch/08834/tg881334/2019"
        self.glob = "NDARINV0BL" #"NDARINV0" #let's look at this particular case (이러면 원래 너무 많을 수도 있따 )
    

In [3]:
args = Args()
file_to_untar = []


last_member_names = []
for tar_dir in [args.T1_DWI_dir]: #[args.T1_DWI_dir]:#[args.T2_dir, args.T1_DWI_dir]:
    tar = tarfile.open(tar_dir)
    for i,member in enumerate(tar):
        file_name = member.name.split('/')[-1]
        if not member.isfile(): #skip if not file
            print("NOT", member)
            continue 
        elif member.name.split('_')[-3] != "baselineYear1Arm1": #skip if baseline
            continue 
        elif args.glob not in file_name: #skip if doesn't match glob 
            continue 
        
        file_to_untar.append(member)
    
        if i > 100000: #임의의 제한.. 굳이 둘 필요 없음!
            break  
    last_member_names.append(member)
    print(tar_dir)

NOT <TarInfo 'NDARINV0' at 0x2b375427be80>
/scratch/08834/tg881334/NDARINV0.tar


In [4]:
last_member_names

[<TarInfo 'NDARINV0/NDARINV00U4FTRU_baselineYear1Arm1_ABCD-MPROC-T1_20180519105135.tgz' at 0x2b3755ba6080>]

In [5]:
print(file_to_untar)
tar.extractall(members = file_to_untar, path = args.save_dir) #extract files


for i,member in enumerate(last_member_names):
    #raise ValueError("muyst be changed")
    upper_dir = os.path.split(member.name)[0] #get uppder dir name
    
    [shutil.move(file, args.save_dir) for file in glob.glob(os.path.join(args.save_dir, f"{upper_dir}/*"))] #move them to the actual directory I wanted

[<TarInfo 'NDARINV0/NDARINV0BL9EL2Y_baselineYear1Arm1_ABCD-MPROC-DTI_20180213110119.tgz' at 0x2b37544a2f80>, <TarInfo 'NDARINV0/NDARINV0BL9EL2Y_baselineYear1Arm1_ABCD-MPROC-T1_20180213103943.tgz' at 0x2b3755b67940>]


In [6]:
os.chdir(args.save_dir)

#unzip the .tgz files
for file in os.listdir(args.save_dir):
    print(file)
    if "tgz" in file: #i.e. if tgz
        subprocess.run(f'tar zxvf {file}', shell = True)
        subprocess.run(f'rm {file}', shell = True)
    elif not "sub-" in file and "dataset_description.json" != file: #i.e. 이미 unzip된 sub-파일이 아니라면, 또는 dataset_description이 아니라면 
        shutil.rmtree(file) #remove the nuisance stuff


NDARINV0
NDARINV0BL9EL2Y_baselineYear1Arm1_ABCD-MPROC-T1_20180213103943.tgz
sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/anat/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_T1w.nii
dataset_description.json
sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/anat/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_T1w.json
NDARINV0BL9EL2Y_baselineYear1Arm1_ABCD-MPROC-DTI_20180213110119.tgz
sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/dwi/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_dwi.nii


tar: sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/anat/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_T1w.json: implausibly old time stamp 1969-12-31 18:00:00


dataset_description.json
sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/dwi/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_dwi.bval
sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/dwi/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_dwi.bvec
sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/dwi/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_dwi.json


tar: sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/dwi/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_dwi.bval: implausibly old time stamp 1969-12-31 18:00:00
tar: sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/dwi/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_dwi.bvec: implausibly old time stamp 1969-12-31 18:00:00
tar: sub-NDARINV0BL9EL2Y/ses-baselineYear1Arm1/dwi/sub-NDARINV0BL9EL2Y_ses-baselineYear1Arm1_run-01_dwi.json: implausibly old time stamp 1969-12-31 18:00:00


In [7]:
raise ValueError("hi")

ValueError: hi

In [ ]:
import os 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import pdb
from pathlib import Path

T2_dir = Path("/work2/08834/tg881334/stampede2/ABCD_preproc/txt_files/T2/")
T1_DWI_dir = Path("/work2/08834/tg881334/stampede2/ABCD_preproc/txt_files/T1_DWI/")

In [ ]:
os.listdir(T2_dir)

In [ ]:
pd.read_csv(T2_dir / "dataset_collection.txt", sep = "\t")
pd.read_csv(T2_dir / "guid_pseudoguid.txt", sep = "\t")
pd.read_csv(T2_dir / "md5_values.txt", sep = "\t")
pd.read_csv(T2_dir / "fmriresults01.txt", sep = "\t") #this is what should be used
pd.read_csv(T2_dir / "package_info.txt", sep = "\t") #this is what should be used

# 0. actually reading the thing 

In [ ]:
T1_DWI_meta = pd.read_csv(T1_DWI_dir / "fmriresults01.txt", sep = "\t", header = 0 )
T2_meta = pd.read_csv(T2_dir / "fmriresults01.txt", sep = "\t", header = 0 ) 


In [ ]:
##removing the first layer because they won't be used (just extra descriptions)
descriptions = T2_meta.iloc[0]
T2_meta = T2_meta.drop(index = 0)
T1_DWI_meta = T1_DWI_meta.drop(index=0)


##seperating T1_DWI_meta into T1 and DWI 
T1_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'MR structural (T1)']
DWI_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'multi-shell DTI']

In [ ]:
##just testing

print(descriptions[0:5].values)
print(T1_meta.shape, T2_meta.shape, DWI_meta.shape)
T1_meta.head()

In [ ]:
##참조
#check/proof that we're only dealing with QC-ed subjects
(T1_meta.qc_outcome != "pass").sum() + (T2_meta.qc_outcome != "pass").sum() + (DWI_meta.qc_outcome != "pass").sum()

# 1. Merging the Datasets

## 1.1 Finding common subjects (which will be used)

# 잠만 : step0에서 그 release별로 구분하는 것도 해야함! (우리는 releae4 만 할것...)
# 따라서 밑에서 overlapping하는 사람들만 빼내는 것도, release4만 빼낸 후에 overlapping을 빼야할듯

In [ ]:
##defining the subject sets
T2_sub_set = set((T2_meta['subjectkey']).values)
T1_sub_set = set((T1_meta['subjectkey']).values)
DWI_sub_set = set((DWI_meta['subjectkey']).values)
print(len(T1_sub_set), len(T2_sub_set), len(DWI_sub_set))

In [ ]:
##getting intersection
total_set = (T2_sub_set.intersection(T1_sub_set)).intersection(DWI_sub_set)
print(len(total_set))

In [ ]:
meta_list = [T1_meta, T2_meta, DWI_meta]

meta = meta_list[0]

haha = meta.loc[meta['subjectkey'].isin(total_set)]
len(haha)
#for meta in meta_list:
    

In [ ]:
set(T1_DWI_meta.scan_type.values)

In [ ]:
T1_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'MR structural (T1)']
DWI_meta = T1_DWI_meta[T1_DWI_meta.scan_type == 'multi-shell DTI']

In [ ]:
#T2_meta.head()
T1_DWI_meta.head()

In [ ]:
#T1_DWI_meta.sort_values('subjectkey')
T1_DWI_sub_set = set((T1_DWI_meta['subjectkey'][1:]).values)
T2_sub_set = set((T2_meta['subjectkey'][1:]).values)

common = T1_DWI_sub_set.intersection(T2_sub_set)
T1_only = T1_DWI_sub_set.intersection(T2_sub_set)
T2_only = 
len(common)

In [ ]:
T2_meta.index #

In [ ]:
T2_meta.interview_age.head()

In [ ]:
T2_meta[(T2_meta['subjectkey'] != T2_meta['src_subject_id'])]


In [ ]:
T1_DWI_meta[(T1_DWI_meta['subjectkey'] != T1_DWI_meta['src_subject_id'])]

In [ ]:
T2_meta.shape

In [ ]:
T2_meta['subjectkey'][::1000]

In [ ]:
T1_DWI_meta['subjectkey'][::1000]

In [ ]:
(T2_meta['qc_outcome'] == "pass").sum()

In [ ]:
set(T2_meta['qc_outcome'].values)